# Test Scan 
## New MongoDB database. April 10, 2017

In [23]:
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import h5py

import pprint

import pymongo
from pymongo import MongoClient

# make graphics inline
%matplotlib inline

In [24]:
client = MongoClient('xf03id1-mdb01', 27017)

In [25]:
ds = client['datastore-new']
fs = client['filestore-new']

## run_start collection

In [26]:
run_starts = ds['run_start']
t1 = datetime.now();
run_start_docs = [doc for doc in run_starts.find({})]
t2 = datetime.now();
print(run_starts.count(), len(run_start_docs), "time:", (t2 - t1), " seconds")

14957 14957 time: 0:00:01.802473  seconds


In [27]:
scan_ids = [doc['scan_id'] for doc in run_start_docs]
max(scan_ids)

47997

In [28]:
run_start_doc = run_start_docs[-1]
print("scan id:", run_start_doc['scan_id'], "time:",
      datetime.fromtimestamp(run_start_doc['time']).strftime("%Y-%m-%d %H:%M:%S.%f%z (%Z)"),
     'uid:', run_start_doc['uid'])

scan id: 47997 time: 2018-04-10 18:37:00.132566 () uid: f638a84f-3c3b-491d-a613-3dd743e8271d


## run_stop collection

In [29]:
run_stops = ds['run_stop']
t1 = datetime.now();
run_stop_docs = [doc for doc in run_stops.find({'run_start' : run_start_doc['uid']})]
t2 = datetime.now();
print(len(run_stop_docs), "time:", (t2 - t1), " seconds")

1 time: 0:00:00.003372  seconds


In [30]:
run_stop_doc = run_stop_docs[-1]
pprint.pprint(run_stop_doc)

{'_id': ObjectId('5acd48bb45e1975c04b21089'),
 'exit_status': 'success',
 'num_events': {'baseline': 2, 'primary': 40000},
 'run_start': 'f638a84f-3c3b-491d-a613-3dd743e8271d',
 'time': 1523402939.0384538,
 'uid': '4976fe05-8e88-4e2a-a952-8c88b762c498'}


In [31]:
datetime.fromtimestamp(run_stop_doc['time']).strftime("%Y-%m-%d %H:%M:%S.%f%z (%Z)")

'2018-04-10 19:28:59.038454 ()'

## event_descriptor collection

In [32]:
event_descriptors = ds['event_descriptor']
t1 = datetime.now();
event_descriptor_docs = [doc for doc in event_descriptors.find({'run_start' : run_start_doc['uid']})]
t2 = datetime.now();
print(len(event_descriptor_docs), "time:", (t2 - t1), " seconds")

2 time: 0:00:00.008561  seconds


In [33]:
for event_descriptor_doc in event_descriptor_docs:
    print(event_descriptor_doc['name'])

primary
baseline


In [34]:
event_descriptor_doc = event_descriptor_docs[0]

## event collection

In [35]:
events = ds['event']
t1 = datetime.now();
event_docs = [doc for doc in events.find({'descriptor' : event_descriptor_doc['uid']})]
t2 = datetime.now();
print(len(event_docs), "time:", (t2 - t1), " seconds")

40000 time: 0:00:04.647445  seconds


In [36]:
merlin1_ids = []
xspress3_ch1_ids = []
xspress3_ch2_ids = []
xspress3_ch3_ids = []
for doc in event_docs:
    merlin1_ids.append(doc['data']['merlin1'])
    xspress3_ch1_ids.append(doc['data']['xspress3_ch1'])
    xspress3_ch2_ids.append(doc['data']['xspress3_ch2'])
    xspress3_ch3_ids.append(doc['data']['xspress3_ch3'])

In [37]:
print(merlin1_ids[0], xspress3_ch1_ids[0], xspress3_ch2_ids[0], xspress3_ch3_ids[0])

fcc1e7f2-1827-40d9-a3d2-31a88e6318c4 16156230-83f8-4efc-9e29-b1aa62369eb6 fd9eecaf-f4f3-460e-84cb-44187b810568 49f5adcd-540f-474b-b3d6-c48f2921eb93


## datum collection

In [38]:
datum = fs['datum']

In [39]:
# check all merlin1_ids
t1 = datetime.now();
merlin1_docs = [doc for doc in datum.find({'datum_id' : {'$in' : merlin1_ids}})]
t2 = datetime.now();
print(len(merlin1_docs), "time:", (t2 - t1), " seconds")

40000 time: 0:00:02.819134  seconds


In [40]:
# check the merlin1 resource ids
merlin1_res_ids = []
for doc in merlin1_docs:
    merlin1_res_ids.append(doc['resource'])
merlin1_res_set = set(merlin1_res_ids)
print(merlin1_res_set)

{'addd794c-3ed9-45a2-a8c6-dd492700da6e', '46e2e921-80d7-44c1-8204-812cc9ea4489', 'f514b0d6-d9dd-4590-b9a6-4b85f15c441d'}


In [41]:
# check all xspress3_ch1_ids
t1 = datetime.now();
xspress3_ch1_docs = [doc for doc in datum.find({'datum_id' : {'$in' : xspress3_ch1_ids}})]
t2 = datetime.now();
print(len(xspress3_ch1_docs), "time:", (t2 - t1), " seconds")

40000 time: 0:00:02.850975  seconds


In [42]:
# check the xspress3_ch1 resource ids
xspress3_ch1_ids = []
for doc in xspress3_ch1_docs:
    xspress3_ch1_ids.append(doc['resource'])
xspress3_ch1_set = set(xspress3_ch1_ids)
print(xspress3_ch1_set)

{'c9d9225b-7479-466f-8f3a-b0adcdc9c845', '25b5a599-2b09-41c8-a97c-5957a7cf9ed6', '16a3bf86-6671-483d-8dd5-aef8f59e2894'}


In [43]:
# check all xspress3_ch2_ids
t1 = datetime.now();
xspress3_ch2_docs = [doc for doc in datum.find({'datum_id' : {'$in' : xspress3_ch2_ids}})]
t2 = datetime.now();
print(len(xspress3_ch2_docs), "time:", (t2 - t1), " seconds")

40000 time: 0:00:02.412624  seconds


In [44]:
# check the xspress3_ch2 resource ids
xspress3_ch2_ids = []
for doc in xspress3_ch2_docs:
    xspress3_ch2_ids.append(doc['resource'])
xspress3_ch2_set = set(xspress3_ch2_ids)
print(xspress3_ch2_set)

{'c9d9225b-7479-466f-8f3a-b0adcdc9c845', '25b5a599-2b09-41c8-a97c-5957a7cf9ed6', '16a3bf86-6671-483d-8dd5-aef8f59e2894'}


In [45]:
# check all xspress3_ch3_ids
t1 = datetime.now();
xspress3_ch3_docs = [doc for doc in datum.find({'datum_id' : {'$in' : xspress3_ch3_ids}})]
t2 = datetime.now();
print(len(xspress3_ch3_docs), "time:", (t2 - t1), " seconds")

40000 time: 0:00:02.749975  seconds


In [46]:
# check the xspress3_ch3 resource ids
xspress3_ch3_ids = []
for doc in xspress3_ch3_docs:
    xspress3_ch3_ids.append(doc['resource'])
xspress3_ch3_set = set(xspress3_ch3_ids)
print(xspress3_ch3_set)

{'c9d9225b-7479-466f-8f3a-b0adcdc9c845', '25b5a599-2b09-41c8-a97c-5957a7cf9ed6', '16a3bf86-6671-483d-8dd5-aef8f59e2894'}


## resource collection

In [47]:
resources = fs['resource']

In [48]:
# get the merlin1 resources
t1 = datetime.now();
merlin1_res_docs = [doc for doc in resources.find({'uid' : {'$in' : list(merlin1_res_set)}})]
t2 = datetime.now();
print(len(merlin1_res_docs), "time:", (t2 - t1), " seconds")

3 time: 0:00:00.052144  seconds


In [49]:
for doc in merlin1_res_docs:
    print(doc['resource_path'])

2018/04/10/603a8495-b4e6-43fa-98f0_000000.h5
2018/04/10/f993b083-3e27-44d9-8241_000000.h5
2018/04/10/4a34d8a4-c467-4439-900c_000000.h5


In [50]:
# get the xspress3_ch1 resources
t1 = datetime.now();
xspress3_ch1_res_docs = [doc for doc in resources.find({'uid' : {'$in' : list(xspress3_ch1_set)}})]
t2 = datetime.now();
print(len(xspress3_ch1_res_docs), "time:", (t2 - t1), " seconds")

3 time: 0:00:00.034254  seconds


In [51]:
for doc in xspress3_ch1_res_docs:
    print(doc['resource_path'])

2018/04/10/1d9b4090-ab09-4d9a-b2e6_000000.h5
2018/04/10/4b7a7ec7-041b-4666-8be1_000000.h5
2018/04/10/27be42b9-6d31-48d9-8787_000000.h5


In [28]:
client.close()